In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import env
import wrangle as w
import os
import matplotlib.pyplot as plt

In [2]:
# acquire Telco churn data
df = w.wrangle_telco_data()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    7043 non-null   object 
 1   senior_citizen            7043 non-null   int64  
 2   partner                   7043 non-null   object 
 3   dependents                7043 non-null   object 
 4   tenure                    7043 non-null   int64  
 5   phone_service             7043 non-null   object 
 6   multiple_lines            7043 non-null   object 
 7   internet_service_type_id  7043 non-null   int64  
 8   tech_support              7043 non-null   object 
 9   contract_type_id          7043 non-null   int64  
 10  payment_type_id           7043 non-null   int64  
 11  monthly_charges           7043 non-null   float64
 12  total_charges             7032 non-null   float64
 13  churn                     7043 non-null   object 
dtypes: float

In [4]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,tech_support,contract_type_id,payment_type_id,monthly_charges,total_charges,churn
0,Female,0,Yes,Yes,9,Yes,No,1,Yes,2,2,65.6,593.30,No
1,Male,0,No,No,9,Yes,Yes,1,No,1,2,59.9,542.40,No
2,Male,0,No,No,4,Yes,No,2,No,1,1,73.9,280.85,Yes
3,Male,1,Yes,No,13,Yes,No,2,No,1,1,98.0,1237.85,Yes
4,Female,1,Yes,No,3,Yes,No,2,Yes,1,2,83.9,267.40,Yes


In [54]:
subset1 = df[['senior_citizen', 'churn']]

In [57]:
pd.crosstab(subset1.senior_citizen, subset1.churn)

churn,No,Yes
senior_citizen,,
0,4508,1393
1,666,476


In [58]:
subset2 = df[['payment_type_id', 'churn']]

In [59]:
subset2

,payment_type_id,churn
0,2,No
1,2,No
2,1,Yes
3,1,Yes
4,2,Yes
...,...,...
7038,2,No
7039,1,Yes
7040,2,No
7041,2,No


In [ ]:
online_security	online_backup	device_protection streaming_tv	streaming_movies paperless_billing

In [7]:
tdf = pd.read_csv('telco_churn.csv')

In [9]:
tdf.shape

(7043, 22)

Lets use a chi-square test to see if Churn and Senior Citizen status are related

I will use a confidence interval of 95%
The resulting alpha is .05

Ho: 'churn' and 'senior_citizen' are independent of one another.
Ha: 'churn and 'senior_citizen' are related.

In [ ]:
H
0
 (Null Hypothesis): class is independent of cylinders.

H
a
 (Alternative Hypothesis): class is dependent on cylinders.